In [7]:
import numpy as np
from scipy import stats, linalg

import matplotlib.pyplot as plt

## Functions

In [2]:
def mchlis_ment(x):
    
    return 0.65*x/(1 + x/20)

def sq_exp(x, σsq):
    
    # initialize covariance
    n = len(x)
    Σ = np.zeros([n,n])
    
    # build covariance
    for i in range(n):
        for j in range(n):
            Σ[i,j] = σsq*np.exp(-((x[i] - x[j])/3)**2)
    
    return Σ

def gls_est(X, y, Σ):
    
    # transpose regressors, nxp -> pxn
    X_T = np.transpose(X)
    
    Σ_inv = np.linalg.solve(Σ,np.identity(2))
    
#     # compute repeated term, (pxn)(nxn) -> pxn
#     Σ_invX = np.solve(Σ,X)
    
#     term1 = np.dot(X_T, Σ_inv)
    
#     assert np.shape(X_TΣ_inv) == (2,5)
#     # compute first term, (pxn)(nxp)->(pxp)
#     X_TΣ_invX = np.dot(X_TΣ_inv, X)
    
#     assert np.shape(X_TΣ_invX) == (2,2)
    
#     # invert first term, (pxp)
#     term1 = np.linalg.inv(X_TΣ_invX)
    
#     # compute second term, (pxn)(nx1)->(px1)
#     term2 = np.dot(X_TΣ_inv, y)
    
#     # compute generalized least squares estimator,(pxp)x(px1)--> (px1)
#     θ_hat = np.dot(term1, term2)
    
    return θ_hat

## Synthetic data generation

In [3]:
# set seed for reproducibility
np.random.seed(234)

## generate synthetic data
x = np.linspace(0, 10, 10).reshape(10,1)

# compute ground truth model
ζ = mchlis_ment(x)

# set ground truth variance of noise
σϵsq_tru = 0.1

# generate normal i.i.d. measurement error, ϵ_1,..., ϵ_n ~iid N(0,σϵsq_tru I_n)
ϵ = stats.norm.rvs(loc = 0, scale = np.sqrt(σϵsq_tru), size = len(x))

# generate observations
y = ζ + ϵ

## Set bounds and initial values for parameter estimation

In [4]:
## variance of discrepancy
# lower bound
σδsq_lo = 1
# upper bound
σδsq_hi = 10
# inital guess
σδsq_0 = 0.1

## variance of noise
# lower bound
σϵsq_lo = 0.01
# upper bound
σϵsq_hi = 2
# inital guess
σϵsq_0 = 0.1

## concatenate bounds and inital guesses
# bounds
bds = [(σδsq_lo, σδsq_hi),(σϵsq_lo, σϵsq_hi)]
# inital guesses
σsq_0 = [(σδsq_0, σϵsq_0)]

## Optimization

In [17]:
p = 3
X = np.tile(x, (1,p))

Σ_δ = sq_exp(X[:,0], σδsq_0)
Σ_ϵ = σϵsq_0*np.identity(len(X[:,0]))
Σ = Σ_δ + Σ_ϵ


# cholesky decomposition of Σ = LL^T 
L = linalg.cholesky(Σ, lower = True)

# solve Lz = y for z
z = np.linalg.solve(L,y)

# solve L wj = xj for wj for all j in 1,..,p
W = X*0
for j in range(len(X[0,:])):
    W[:,j] = np.linalg.solve(L, X[:,j])


# ## compute first term, (pxn)(nxp)->(pxp)

# # solve system (nxn)(nxp) -> (nxp)
# Σ_invX = np.linalg.solve(Σ, X)
# print('Σ^-1 X \n',np.shape(Σ_invX))

# # transpose regressors, (nxp) -> (pxn)
# X_T = np.transpose(X)
# print('Σ^T \n', np.shape(X_T))

# # compute first term, (pxn)(nxp)->(pxp)
# X_TΣ_invX = np.dot(X_T, Σ_invX)
# print('X^T Σ_^-1 X \n', np.shape(X_TΣ_invX)) 
  
    
# ## compute second term
# # (nxn)(nx1)->(nx1)
# Σ_invy = np.linalg.solve(Σ,y)
# print('Σ_^-1 y \n', np.shape(Σ_invy)) 

# # (pxn)(nx1)-> (px1)
# X_TΣ_invy = np.dot(X_T, Σ_invy)
# print('X_T Σ_^-1 y \n', np.shape(X_TΣ_invy))

# ## compute gls
# # (pxp)(px1)->(px1)
# θ_hat = np.linalg.solve(X_TΣ_invX, X_TΣ_invy)
# print('θ_hat \n', np.shape(θ_hat))

# #θ = gls_est(X, y, Σ)

(10, 10)
